# Model Project: The Agents Labour Supply Decision


In present project we seek to model the supply of labour by studying a two-good static problem, where agents value leisure and consumption. As an extension for the base model we impose a progressive tax system, which creates kinks in the budget constraint. We then show, how this may create bunching around tax bracket cut-offs.

The basic problem of the agents is:


$$\begin{aligned}  \max_{c,l}\ \ &u(c,l) \quad s.t. \ \ R_0 = wl + c  \quad c,l\geq0\\
 &\dfrac{\partial u}{\partial a}>0 \quad  \wedge \quad \dfrac{\partial^2 u}{\partial a^2}<0,  \qquad a=c,l \\
 &R_0=wT + A_0 
 \end{aligned} $$
 

Where $R_0$ is the income potential of the agent, $c$ the consumption which is a numeraire, $l$ is the amount of leisure consumed with the associated oppurtunity cost of being the wage rate, $w$. We assume marginal utility of consumption and leisure are positive but decreasing. Further we for now assume the income potential is the sum of wage earnings from working your entire time endowment, $T$, and non-labour income, $A_0$. The problem is framed such that the agent at default suplies all his time as labour giving her/him an income of $R_0$. The agent now has to decide how much leisure she/he will 'buy back', considering the consumption potential which is implied.

Dependencies:

In [ ]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy.optimize import Bounds
import sympy as sm

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

We will employ *SymPy* to solve the basic model: